In [1]:
import pandas as pd
import os
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pickle

#### Import Data

In [2]:
# path to processed data
PROCESSED_DATA_FP = os.path.join(os.path.pardir,'data', 'processed')
TRAIN_FN = os.path.join(PROCESSED_DATA_FP, 'train.csv')
TEST_FN = os.path.join(PROCESSED_DATA_FP, 'test.csv')

In [3]:
train_df = pd.read_csv(TRAIN_FN, index_col='PassengerId')
test_df = pd.read_csv(TEST_FN, index_col='PassengerId')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            891 non-null    float64
 1   Age                 891 non-null    float64
 2   Fare                891 non-null    float64
 3   Family_Size         891 non-null    int64  
 4   Is_Mother           891 non-null    int64  
 5   Is_Male             891 non-null    int64  
 6   Is_Female           891 non-null    int64  
 7   Deck_A              891 non-null    int64  
 8   Deck_B              891 non-null    int64  
 9   Deck_C              891 non-null    int64  
 10  Deck_D              891 non-null    int64  
 11  Deck_E              891 non-null    int64  
 12  Deck_F              891 non-null    int64  
 13  Deck_G              891 non-null    int64  
 14  Deck_Z              891 non-null    int64  
 15  Pclass_1            891 non-null    int64  
 16  Pclass_2     

In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 418 non-null    float64
 1   Fare                418 non-null    float64
 2   Family_Size         418 non-null    int64  
 3   Is_Mother           418 non-null    int64  
 4   Is_Male             418 non-null    int64  
 5   Is_Female           418 non-null    int64  
 6   Deck_A              418 non-null    int64  
 7   Deck_B              418 non-null    int64  
 8   Deck_C              418 non-null    int64  
 9   Deck_D              418 non-null    int64  
 10  Deck_E              418 non-null    int64  
 11  Deck_F              418 non-null    int64  
 12  Deck_G              418 non-null    int64  
 13  Deck_Z              418 non-null    int64  
 14  Pclass_1            418 non-null    int64  
 15  Pclass_2            418 non-null    int64  
 16  Pclass_3  

#### Data Preparation

In [6]:
# Extract features from 'Age' onwards, convert to NumPy array, and cast to float
X = train_df.loc[:, 'Age':].to_numpy().astype('float')

# Extract target variable ('Survived') and flatten to 1D array
y = train_df['Survived'].values.ravel()


In [7]:
# Display shapes of feature and target arrays to verify correct dimensions
print(X.shape, y.shape)

(891, 35) (891,)


In [8]:
# train test split
# split 20% for test data, random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(712, 35) (712,)
(179, 35) (179,)


In [9]:
# get proportion of positive classes
# '1' meaning the passenger survived
# the mean should be close to each other between train and test cases
print(f"Mean survival in train: {np.mean(y_train):.3f}")
print(f"Mean survival in test: {np.mean(y_test):.3f}")

Mean survival in train: 0.376
Mean survival in test: 0.413


#### Build Training Model

In [10]:
# Create classification training model
model_dummy = DummyClassifier(strategy='most_frequent', random_state=42)

In [11]:
# train model
# input data and output data
model_dummy.fit(X_train, y_train)

DummyClassifier(random_state=42, strategy='most_frequent')

In [12]:
print(f'Score for training model: {model_dummy.score(X_test, y_test):.2f}')

Score for training model: 0.59


In [13]:
# Accuracy Score
print(f'Accuracy for training model: {accuracy_score(y_test, model_dummy.predict(X_test)):.2f}')

Accuracy for training model: 0.59


In [14]:
# confusion matrix
print(f'Confusion Matrix for training model:\n {confusion_matrix(y_test, model_dummy.predict(X_test))}')

Confusion Matrix for training model:
 [[105   0]
 [ 74   0]]


In [15]:
# Precision score
print(f'Precision for training model:\n {precision_score(y_test, model_dummy.predict(X_test))}')
# recall score
print(f'Recall for training model:\n {recall_score(y_test, model_dummy.predict(X_test))}')

Precision for training model:
 0.0
Recall for training model:
 0.0


C:\Users\james\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### First Kaggle Submission

In [16]:
# Convert to the numpy array
test_X = test_df.to_numpy().astype('float')

In [17]:
# get_predictions
predictions = model_dummy.predict(test_X)

In [18]:
# create submission dataframe
df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})

In [19]:
df_submission

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,0.0
...,...,...
413,1305,0.0
414,1306,0.0
415,1307,0.0
416,1308,0.0


In [20]:
# create filepath variables
submission_data_fp = os.path.join(os.path.pardir, 'data', 'external')
submission_data_fn = os.path.join(submission_data_fp, '01_dummy.csv')

In [21]:
# save submission_df
df_submission.to_csv(submission_data_fn, index=False)

In [22]:
# Function to get submission
def get_submission_file(model, filename):
    # Convert to the numpy array
    test_X = test_df.to_numpy().astype('float')
    # get_predictions
    predictions = model_dummy.predict(test_X)
    # create submission dataframe
    df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})
    # create filepath variables
    submission_data_fp = os.path.join(os.path.pardir, 'data', 'external')
    submission_data_fn = os.path.join(submission_data_fp, filename)
    # save submission_df
    df_submission.to_csv(submission_data_fn, index=False)

In [23]:
# call get submission file function
get_submission_file(model_dummy, '01_dummy.csv')

In [24]:
# create LR Model
model_lr_1 = LogisticRegression(max_iter=2000, random_state=42)

In [25]:
# train LR Model
model_lr_1.fit(X_train, y_train)

LogisticRegression(max_iter=2000, random_state=42)

In [26]:
# get model parameters
print(model_lr_1.get_params())

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 2000, 'multi_class': 'deprecated', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [27]:
# Get the actual number of iterations used by the model
print(f"Number of iterations to converge: {model_lr_1.n_iter_}")

Number of iterations to converge: [643]


In [28]:
# evaluate model
print(f'Score for Logistic Regression - Version 1: {model_lr_1.score(X_test, y_test):.2f}')

Score for Logistic Regression - Version 1: 0.82


In [29]:
# Accuracy Score
print(f'Accuracy for Logistic Regression - Version 1: \n {accuracy_score(y_test, model_lr_1.predict(X_test)):.2f}')
# confusion matrix
print(f'Confusion Matrix for Logistic Regression - Version 1:\n {confusion_matrix(y_test, model_lr_1.predict(X_test))}')
# Precision score
print(f'Precision for Logistic Regression - Version 1:\n {precision_score(y_test, model_lr_1.predict(X_test)):.2f}')
# recall score
print(f'Recall for Logistic Regression - Version 1:\n {recall_score(y_test, model_lr_1.predict(X_test)):.2f}')

Accuracy for Logistic Regression - Version 1: 
 0.82
Confusion Matrix for Logistic Regression - Version 1:
 [[88 17]
 [16 58]]
Precision for Logistic Regression - Version 1:
 0.77
Recall for Logistic Regression - Version 1:
 0.78


In [30]:
# model coefficients
model_lr_1.coef_

array([[-0.02822519,  0.00522526, -0.55909453,  0.18796416, -0.88617778,
         0.88858937, -0.25462345,  0.33446279, -0.07415635,  0.70665225,
         1.15687922, -0.04023613, -0.99386973, -0.83269702, -0.05524132,
         0.51660616, -0.45895325,  0.19752573,  1.22992165, -0.26739289,
        -1.15403794,  0.95845652, -0.09940029, -0.8626612 , -0.17878076,
        -0.24563752,  0.07231917,  0.3545107 ,  0.25824144,  0.02969332,
        -0.28552318, -0.17365469,  0.32462529,  0.3931824 , -0.54174141]])

In [31]:
# call get submission file function
get_submission_file(model_lr_1, '02_lr.csv')

#### Hyperparameters Optimization: Grid Search
##### Utilize k-fold Cross Validation

In [32]:
# create LR Model
model_lr = LogisticRegression(solver='liblinear', max_iter=2000, random_state=42)

In [33]:
parameters = {
    'C': [1.0, 10.0, 50.1, 100.0, 1000.0],
    'penalty': ['l1', 'l2']
}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=5)

In [34]:
# train GridSearch LR models
clf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(max_iter=2000, random_state=42,
                                          solver='liblinear'),
             param_grid={'C': [1.0, 10.0, 50.1, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [35]:
# to get best settings
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [36]:
print(f'best score : {clf.best_score_}')

best score : 0.8314389835516597


In [37]:
# get test model score
print(f'Score for Logistic Regression - Version 2: {clf.score(X_test, y_test):.2f}')

Score for Logistic Regression - Version 2: 0.82


In [38]:
# call get submission file function
get_submission_file(clf, '03_lr.csv')

#### Feature Normalization and Standardization

##### Feature Normalization

In [39]:
# Feature Normalization with MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [40]:
# normalize test data
# Check the minimum and maximum values of the first feature in the normalized test data
X_train_scaled[:, 0].min(), X_train_scaled[:, 0].max()

(0.0, 1.0)

In [41]:
# Create LR model with GridSearchCV
model_lr = LogisticRegression(solver='liblinear', max_iter=2000, random_state=42)
parameters = {
    'C': [1.0, 10.0, 50.1, 100.0, 1000.0],
    'penalty': ['l1', 'l2']
}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=5)
clf.fit(X_train_scaled, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(max_iter=2000, random_state=42,
                                          solver='liblinear'),
             param_grid={'C': [1.0, 10.0, 50.1, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [42]:
# to get best settings
clf.best_score_

0.8244361272530287

In [43]:
# get test model score
print(f'Score for Logistic Regression - Version 2: {clf.score(X_test_scaled, y_test):.2f}')

Score for Logistic Regression - Version 2: 0.82


##### Feature Standardization

In [44]:
# Feature Standardization with StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##### Create model after Standardization

In [45]:
# Create LR model with GridSearchCV
model_lr = LogisticRegression(solver='liblinear', max_iter=2000, random_state=42)
parameters = {
    'C': [1.0, 10.0, 50.1, 100.0, 1000.0],
    'penalty': ['l1', 'l2']
}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=5)
clf.fit(X_train_scaled, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(max_iter=2000, random_state=42,
                                          solver='liblinear'),
             param_grid={'C': [1.0, 10.0, 50.1, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [46]:
# to get best settings
clf.best_score_

0.825844577957254

In [47]:
# get test model score
print(f'Score for Logistic Regression - Version 2: {clf.score(X_test_scaled, y_test):.2f}')

Score for Logistic Regression - Version 2: 0.80


#### Model Persistance

In [48]:
# Define file paths for saving the model and scaler
model_fp = os.path.join(os.path.pardir, 'models', 'lr_model.pkl')
scaler_fp = os.path.join(os.path.pardir, 'models', 'lr_scaler.pkl')

In [49]:
# Open files in write-binary mode
model_file_pickle = open(model_fp, 'wb')
scaler_file_pickle = open(scaler_fp, 'wb')

In [50]:
# persist the model and scaler
pickle.dump(clf, model_file_pickle)
pickle.dump(scaler, scaler_file_pickle)

In [51]:
# Open files in read-binary mode to load the model and scaler
model_file_pickle = open(model_fp, 'rb')
scaler_file_pickle = open(scaler_fp, 'rb')
# Load the model and scaler from the files
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded= pickle.load(scaler_file_pickle)
# Close files after loading
model_file_pickle
scaler_file_pickle

<_io.BufferedReader name='..\\models\\lr_scaler.pkl'>

In [52]:
# Output loaded model
clf_loaded

GridSearchCV(cv=5,
             estimator=LogisticRegression(max_iter=2000, random_state=42,
                                          solver='liblinear'),
             param_grid={'C': [1.0, 10.0, 50.1, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [53]:
# Output loaded scaler
scaler_loaded

StandardScaler()